In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"

# =================== Hyperparameters ===================  
block_size = 8
batch_size = 4
max_iters = 200000
# eval_interval = 2500
learning_rate = 3e-3
eval_iters = 1000
n_embd = 384
n_layer = 4
n_head = 4
# ======================================================= 

device

'cuda'

In [2]:
with open("wizard_of_oz.txt", "r", encoding="utf-8") as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
vocab_size

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


81

In [6]:
# Tokenizers 
string_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_string = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: "".join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
data[:100]

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])

In [7]:
# Doing train/test split
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[60, 58, 57,  1, 73, 61, 58,  1],
        [74, 71,  1, 73, 54, 65, 65,  1],
        [ 9,  1, 66, 78,  1, 57, 58, 54],
        [ 1, 33, 73,  1, 66, 58, 54, 67]], device='cuda:0')
targets:
tensor([[58, 57,  1, 73, 61, 58,  1, 67],
        [71,  1, 73, 54, 65, 65,  1, 72],
        [ 1, 66, 78,  1, 57, 58, 54, 71],
        [33, 73,  1, 66, 58, 54, 67, 73]], device='cuda:0')


In [8]:
# Creating the block
class Block(nn.Module):
    """ Transformer block: communication followed """
    def __init__(self, n_embd, n_head) -> None:
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)

        return x



# Creating the model
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size, n_embd) -> None:
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # Final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def __init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.2)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        # idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index
    
model = BigramLanguageModel(vocab_size=vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
generated_chars


'\nBYxb9EholGQ!6Pq3v-e(B26rPMbbXT\'L&Z05qq]-tDpdrqPE*Gpr1.8pKOvv-?SBY)8&0rHC1YjzxU0zZnvc8Ve"7SewT3IwI8 hc-e\'vQ\'v!tNB2?ASX&m]&"LHyyd\ufeff?sHFWnS_\'kO17cEHtHDUX&5ypTcZd:QF\ufeff?VWKw3oa;o?"X\nKqMlY2pazIEyjzg1RBJVSTl,jc;8VmPh.z6:AXXIHk&h:mrpR(dTHO5x*Ve\nyi]LAHxa&PNWqvI(SNducwLHgsE"n4&eMn7l\'? h66&Z[snb\n]iiPpHDT4 gGe8kkaXO16\ufeffgKP?:XLp6NkyVI"LN2?AA9;L2fmaz0sI[SyJwZ!9oE]krw -PO !0pqi]x71mG"oO&f?F)*k\ufeffM\ufeffw.U-l32?e\nL"bf]NvG0i6J_)QrvD24S_QrcBYP9q?scmjE2"eVaH;m? \ufeff&gb3BoJqR0bNoRfroMa?DmBaU3BE,vqs:2!bd\ufeffna_hrxiAPo?od46\nvj&m!9"N'

In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
# Optimizing the model
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # Get train batches
    xb, yb = get_batch("train")

    # Loss 
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

loss.item()